In [1]:
import pyspark
from pyspark.sql import SparkSession
# REDIS CONFIGURATION
redis_host = "redis"
redis_port = "6379"
spark = SparkSession \
    .builder \
    .master("local") \
    .appName('jupyter-pyspark') \
      .config("spark.redis.host", redis_host)\
      .config("spark.redis.port", redis_port)\
      .config("spark.jars.packages","com.redislabs:spark-redis_2.12:3.1.0")\
    .getOrCreate()
sc = spark.sparkContext
sc.setLogLevel("ERROR")

Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
com.redislabs#spark-redis_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-3d0ca622-bf9f-4d0d-b3a5-e8a4230d9776;1.0
	confs: [default]


:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found com.redislabs#spark-redis_2.12;3.1.0 in central
	found redis.clients#jedis;3.9.0 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
downloading https://repo1.maven.org/maven2/com/redislabs/spark-redis_2.12/3.1.0/spark-redis_2.12-3.1.0.jar ...
	[SUCCESSFUL ] com.redislabs#spark-redis_2.12;3.1.0!spark-redis_2.12.jar (55ms)
downloading https://repo1.maven.org/maven2/redis/clients/jedis/3.9.0/jedis-3.9.0.jar ...
	[SUCCESSFUL ] redis.clients#jedis;3.9.0!jedis.jar (153ms)
downloading https://repo1.maven.org/maven2/org/slf4j/slf4j-api/1.7.30/slf4j-api-1.7.30.jar ...
	[SUCCESSFUL ] org.slf4j#slf4j-api;1.7.30!slf4j-api.jar (20ms)
downloading https://repo1.maven.org/maven2/org/apache/commons/commons-pool2/2.11.1/commons-pool2-2.11.1.jar ...
	[SUCCESSFUL ] org.apache.commons#commons-pool2;2.11.1!commons-pool2.jar (29ms)
:: resolution report :: resolve 1646ms :: artifacts dl 261ms
	:: modules in use:
	com.redislabs#spark-redis_2

In [2]:
es = spark.read.options(header=True, inferSchema=True)\
    .csv("file:///home/jovyan/datasets/exam-scores/*.csv")
es.createOrReplaceTempView("examscores")

In [3]:
# cReate your own ID
query = '''
select 
row_number() over (order by Class_Section) + 1000 as rowid,
* 
from examscores
'''
es2 = spark.sql(query)
es2.printSchema()

root
 |-- rowid: integer (nullable = true)
 |-- Class_Section: string (nullable = true)
 |-- Exam_Version: string (nullable = true)
 |-- Completion_Time: integer (nullable = true)
 |-- Made_Own_Study_Guide: string (nullable = true)
 |-- Did_Exam_Prep Assignment: string (nullable = true)
 |-- Studied_In_Groups: string (nullable = true)
 |-- Student_Score: integer (nullable = true)
 |-- Percentage: string (nullable = true)
 |-- Letter_Grade: string (nullable = true)



In [4]:
# write to redis with redis generating key
es2.write.format("org.apache.spark.sql.redis")\
  .mode("overwrite")\
  .option("table", "examscores")\
  .save()

In [5]:
# write to redis using specified key
es2.write.format("org.apache.spark.sql.redis")\
  .mode("overwrite")\
  .option("table", "examscores2")\
  .option("key.column", "rowid")\
  .save()

In [6]:
# this looks the same as below
es3 = spark.read.format("org.apache.spark.sql.redis")\
  .option("table", "examscores2")\
  .option("key.column", "rowid").load()
es3.toPandas()

,rowid,Class_Section,Exam_Version,Completion_Time,Made_Own_Study_Guide,Did_Exam_Prep Assignment,Studied_In_Groups,Student_Score,Percentage,Letter_Grade
0,1033,M02,A,35,?,?,?,22,73.30%,C+
1,1011,M01,B,25,?,?,?,20,66.70%,C
2,1065,M02,D,60,N,N,Y,24,80.00%,B
3,1021,M01,C,60,?,?,?,22,73.30%,C+
4,1032,M02,A,30,N,Y,Y,24,80.00%,B
...,...,...,...,...,...,...,...,...,...,...
60,1013,M01,B,35,Y,N,Y,29,96.70%,A
61,1057,M02,C,60,N,N,Y,16,53.30%,D
62,1035,M02,A,45,Y,Y,N,24,80.00%,B
63,1045,M02,B,45,Y,Y,Y,25,83.30%,B


In [7]:
# this looks the same as above
es4 = spark.read.format("org.apache.spark.sql.redis")\
  .option("table", "examscores")\
  .load()
es4.toPandas()

,rowid,Class_Section,Exam_Version,Completion_Time,Made_Own_Study_Guide,Did_Exam_Prep Assignment,Studied_In_Groups,Student_Score,Percentage,Letter_Grade
0,1023,M01,D,25,?,?,?,24,80.00%,B
1,1061,M02,D,40,?,?,?,23,76.70%,B-
2,1057,M02,C,60,N,N,Y,16,53.30%,D
3,1030,M02,A,20,N,N,N,16,53.30%,D
4,1002,M01,A,20,?,?,?,27,90.00%,A-
...,...,...,...,...,...,...,...,...,...,...
60,1050,M02,C,30,N,Y,Y,24,80.00%,B
61,1034,M02,A,40,?,?,?,27,90.00%,A-
62,1003,M01,A,30,Y,Y,Y,30,100.00%,A
63,1048,M02,B,60,N,N,Y,22,73.30%,C+


In [8]:
# this reads the hash
es3 = spark.read.format("org.apache.spark.sql.redis")\
  .option("table", "examscores")\
  .option("key.column", "whateve").load()
es3.printSchema()

root
 |-- rowid: integer (nullable = true)
 |-- Class_Section: string (nullable = true)
 |-- Exam_Version: string (nullable = true)
 |-- Completion_Time: integer (nullable = true)
 |-- Made_Own_Study_Guide: string (nullable = true)
 |-- Did_Exam_Prep Assignment: string (nullable = true)
 |-- Studied_In_Groups: string (nullable = true)
 |-- Student_Score: integer (nullable = true)
 |-- Percentage: string (nullable = true)
 |-- Letter_Grade: string (nullable = true)



In [9]:
from pyspark.sql.functions import *
es_summary = es4.groupBy().agg( 
    min(col("Student_Score")).alias("min_score"), 
    avg(col("Student_Score")).alias("avg_score"), 
    max(col("Student_Score")).alias("max_score") 
                 )

es_summary.write.format("org.apache.spark.sql.redis")\
  .mode("overwrite")\
  .option("table", "examsummary")\
  .save()

In [10]:
es_summary.explain()

== Physical Plan ==
*(2) HashAggregate(keys=[], functions=[min(Student_Score#144), avg(cast(Student_Score#144 as bigint)), max(Student_Score#144)])
+- Exchange SinglePartition, ENSURE_REQUIREMENTS, [id=#131]
   +- *(1) HashAggregate(keys=[], functions=[partial_min(Student_Score#144), partial_avg(cast(Student_Score#144 as bigint)), partial_max(Student_Score#144)])
      +- *(1) Scan org.apache.spark.sql.redis.RedisSourceRelation@47b18dfb [Student_Score#144] PushedFilters: [], ReadSchema: struct<Student_Score:int>




In [14]:
es4.filter("rowid=1023").sort("Completion_Time").explain()

== Physical Plan ==
*(2) Sort [Completion_Time#140 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(Completion_Time#140 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [id=#169]
   +- *(1) Filter (isnotnull(rowid#137) AND (rowid#137 = 1023))
      +- *(1) Scan org.apache.spark.sql.redis.RedisSourceRelation@47b18dfb [rowid#137,Class_Section#138,Exam_Version#139,Completion_Time#140,Made_Own_Study_Guide#141,Did_Exam_Prep Assignment#142,Studied_In_Groups#143,Student_Score#144,Percentage#145,Letter_Grade#146] PushedFilters: [IsNotNull(rowid), EqualTo(rowid,1023)], ReadSchema: struct<rowid:int,Class_Section:string,Exam_Version:string,Completion_Time:int,Made_Own_Study_Guid...




In [15]:
es4.filter("rowid=1023").explain()

== Physical Plan ==
*(1) Filter (isnotnull(rowid#137) AND (rowid#137 = 1023))
+- *(1) Scan org.apache.spark.sql.redis.RedisSourceRelation@47b18dfb [rowid#137,Class_Section#138,Exam_Version#139,Completion_Time#140,Made_Own_Study_Guide#141,Did_Exam_Prep Assignment#142,Studied_In_Groups#143,Student_Score#144,Percentage#145,Letter_Grade#146] PushedFilters: [IsNotNull(rowid), EqualTo(rowid,1023)], ReadSchema: struct<rowid:int,Class_Section:string,Exam_Version:string,Completion_Time:int,Made_Own_Study_Guid...


